In [1]:
# extract images from video - dinosaur
import cv2
vid_path = './data/dinosaur_model.mp4'
vidcap = cv2.VideoCapture(vid_path)

fps = vidcap.get(cv2.CAP_PROP_FPS)
print(fps)
success,image = vidcap.read()
count = 0
im_list = []
while success:
    success,image = vidcap.read()
    if count//fps > 16:
        break
    if count//fps > 9:
        im_list.append(image)
        cv2.imwrite("my_data/part2.3/dinosaur/frame%d.jpg" % count, image)     # save frame as JPEG file
    count += 1

29.97002997002997


In [2]:
# extract images from video - Anaelle
from PIL import Image 
import numpy as np
# extract images from video anaelle
vid_path = './my_data/part2.3/us_video/anaelle_video2.mp4'
vidcap = cv2.VideoCapture(vid_path)
fps = vidcap.get(cv2.CAP_PROP_FPS)
print(fps)
success,image = vidcap.read()
count = 100
im_list = []
while success:
    success,image = vidcap.read()
    if success:
        im_pil = Image.fromarray(image)
        im_pil = im_pil.rotate(-90, expand=True)
        if count//fps > 16:
            break
        image = np.array(im_pil)
        im_list.append(image)

        cv2.imwrite("my_data/part2.3/Anaelle/%d.jpg" % count, image)     # save frame as JPEG file
        count += 1

30.005917197411705


In [3]:
from PIL import Image 
import numpy as np
# extract images from video - Michael
vid_path = './my_data/part2.3/us_video/michael_video.mp4'
vidcap = cv2.VideoCapture(vid_path)
fps = vidcap.get(cv2.CAP_PROP_FPS)
print(fps)
success,image = vidcap.read()
count = 100
im_list = []
while success:
    success,image = vidcap.read()
    if success:
        im_pil = Image.fromarray(image)
        im_pil = im_pil.rotate(-90, expand=True)
        if count//fps > 16:
            break
        image = np.array(im_pil)
        im_list.append(image)

        cv2.imwrite("my_data/part2.3/Michael/%d.jpg" % count, image)     # save frame as JPEG file
        count += 1

30.04028054878332


In [4]:
#prepare arguments for main loop
import os
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
from part1.segment import calc_masked_image, calc_mask
from part1.segment import segment_deeplabv3
from part1.utils import resize_and_save
import numpy as np
import cv2 as cv

model = torch.hub.load('pytorch/vision:v0.4.0', 'deeplabv3_resnet101', pretrained=True)
model.eval()
lower_bound_fgd = (50, 200, 0)
upper_bound_fgd = (80, 255, 20)
target = Image.open('./my_data/part2_background/jp_large.jpg')

Using cache found in C:\Users\micha/.cache\torch\hub\pytorch_vision_v0.4.0


In [5]:
im_dir_dinosaur = 'my_data/part2.3/dinosaur'
im_dir_Anaelle = 'my_data/part2.3/Anaelle'
im_dir_Michael = 'my_data/part2.3/Michael'
im_dir_masks_anaelle = 'my_data/part2.3/anaelle_masks'
im_dir_masks_michael = 'my_data/part2.3/michael_masks'
im_dir_seg_din = 'my_data/part2.3/din_seg'
im_dir_seg_anaelle = 'my_data/part2.3/anaelle_seg'
im_dir_seg_michael = 'my_data/part2.3/michael_seg'
output_vid_images = 'my_data/part2.3/output_video_images'
num_im =360

for i, (im_dinosaur_name, im_anaelle_name, im_michael_name) in enumerate(zip(sorted(os.listdir(im_dir_dinosaur))[:num_im], sorted(os.listdir(im_dir_Anaelle))[:num_im], sorted(os.listdir(im_dir_Michael))[:num_im])):

    im_dinosaur_path = os.path.join(im_dir_dinosaur, im_dinosaur_name)
    
    mask_dinosaur = calc_mask(im_dinosaur_path, lower_bound_fgd, upper_bound_fgd, in_range=False,is_erode=True)
    # save dinosaur segmented image
    masked_din_im = cv.imread(im_dinosaur_path)
    masked_din_im = cv.cvtColor(masked_din_im, cv.COLOR_BGR2RGB)
    
    seg_din_path = os.path.join(im_dir_seg_din, im_dinosaur_name)
    seg_din_im = masked_din_im * mask_dinosaur[:, :, np.newaxis]
    seg_din_im_pil = Image.fromarray(np.uint8(seg_din_im))
    seg_din_im_pil.save(seg_din_path)
    
    # insert dinasour to image
    masked_im_with_dinosaur = calc_masked_image(im_dinosaur_path, target, shift_down=260, shift_right=100, mask=mask_dinosaur)

    # insert Anaelle to image
    im_anaelle_path = os.path.join(im_dir_Anaelle, im_anaelle_name)
    resize_and_save(im_anaelle_path, 600, 300)
    mask_anaelle_path = os.path.join(im_dir_masks_anaelle, im_anaelle_name)
    seg_anaelle_path = os.path.join(im_dir_seg_anaelle, im_anaelle_name)
    
    if not os.path.isfile(mask_anaelle_path):
        mask_anaelle_tensor, seg_anaelle_im = segment_deeplabv3(model, im_anaelle_path,is_erode=True)
        kernel = np.ones((7, 7), np.uint8)
        # save mask
        mask_anaelle_im_erode = cv2.erode(mask_anaelle_tensor.byte().cpu().numpy(), kernel, iterations=1)
        mask_anaelle_im_erode_pil = Image.fromarray(np.uint8(mask_anaelle_im_erode*255))
        mask_anaelle_im_erode_pil.save(mask_anaelle_path)

        # save segmented image
        seg_anaelle_im = seg_anaelle_im * mask_anaelle_im_erode[:, :, np.newaxis]
        seg_anaelle_im_pil = Image.fromarray(np.uint8(seg_anaelle_im))
        seg_anaelle_im_pil.save(seg_anaelle_path)
    
    else:
        mask_anaelle_im_erode = np.array(Image.open(mask_anaelle_path))/255

        
    masked_im_with_dino_anaelle = calc_masked_image(im_anaelle_path, masked_im_with_dinosaur, shift_down=450, shift_right=500, mask=mask_anaelle_im_erode)
       
    # insert Michael to image
    im_michael_path = os.path.join(im_dir_Michael, im_michael_name)
    resize_and_save(im_michael_path, 400, 200)
    
    mask_michael_path = os.path.join(im_dir_masks_michael, im_michael_name)
    seg_michael_path = os.path.join(im_dir_seg_michael, im_michael_name)
    
    if not os.path.isfile(mask_michael_path):
        mask_michael_tensor, seg_michael_im = segment_deeplabv3(model, im_michael_path,is_erode=True)
        kernel = np.ones((7, 7), np.uint8)
        # save mask
        mask_michael_im_erode = cv2.erode(mask_michael_tensor.byte().cpu().numpy(), kernel, iterations=1)
        mask_michael_im_erode_pil = Image.fromarray(np.uint8(mask_michael_im_erode*255))
        mask_michael_im_erode_pil.save(mask_michael_path)
        # save segmented image
        seg_michael_im = seg_michael_im * mask_michael_im_erode[:, :, np.newaxis]
        seg_michael_im_pil = Image.fromarray(np.uint8(seg_michael_im))
        seg_michael_im_pil.save(seg_michael_path)

    else:
        mask_michael_im_erode = np.array(Image.open(mask_michael_path))/255
#         print(np.unique(mask_our_im))
        
    masked_im_with_dino_anaelle_michael = calc_masked_image(im_michael_path, masked_im_with_dino_anaelle, shift_down=660, shift_right=800, mask=mask_michael_im_erode)
    
    output_vid_im_path = os.path.join(output_vid_images, im_anaelle_name)
    masked_im_with_dino_anaelle_michael.save(output_vid_im_path)


C:\Users\micha\Desktop\cv_hw3\code\part1\segment.py:47: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[insertHere] = array


In [7]:
from frame_video_convert import image_seq_to_video_changed
image_seq_to_video_changed(imgs_path='./my_data/part2.3/output_video_images',num_images=209,output_path='../output/video.mp4', fps=30)

(1600, 1059)
writing video...
saved video @  ../output/video.mp4
